In [ ]:
import datasets
from transformers import AutoTokenizer, AutoModelForTokenClassification

raw_dataset = datasets.load_dataset('eriktks/conll2003')
label_names = raw_dataset['train'].features['ner_tags'].feature.names

num_labels = len(label_names)
id2label = {i:lbl for i, lbl in enumerate(label_names)}
label2id = {lbl:i for i, lbl in enumerate(label_names)}

/home/sam/SeneceAssignments/Semester5/Text Mining/auto-ner/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Using the latest cached version of the dataset since eriktks/conll2003 couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'default' at /home/sam/.cache/huggingface/datasets/eriktks___conll2003/default/0.0.0/ce85b39f9dd99f552d0739d456814e95fb6a39b0 (last modified on Wed Nov 26 22:37:46 2025).


In [2]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')


def preprocess(batch):
    tokenized_inputs = tokenizer(
        batch['tokens'],
        truncation=True,
        is_split_into_words=True,
        padding=False
    )

    labels = []


    for i, label in enumerate(batch['ner_tags']):
        word_ids = tokenized_inputs.word_ids(batch_index=i)

        previous_id = None

        label_ids = []

        for word_id in word_ids:
            if word_id is None:
                label_ids.append(-100)
            elif word_id != previous_id:
                label_ids.append(label[word_id])
            else:
                label_ids.append(-100)
            previous_id = word_id

        labels.append(label_ids)


    tokenized_inputs['labels'] = labels
    return tokenized_inputs

tokenized_dataset = raw_dataset.map(
    preprocess,
    batched=True
)
 
tokenized_dataset

Map: 100%|██████████| 3250/3250 [00:00<00:00, 15273.80 examples/s]


DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 3453
    })
})

In [11]:
raw_dataset['train'].features['ner_tags'].feature.names

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

In [3]:
print(tokenized_dataset['train']['labels'][0])
print(tokenized_dataset['train']['tokens'][0])
print(tokenized_dataset['train']['input_ids'][0])


[-100, 3, 0, 7, 0, 0, 0, 7, 0, 0, -100]
['EU', 'rejects', 'German', 'call', 'to', 'boycott', 'British', 'lamb', '.']
[101, 7327, 19164, 2446, 2655, 2000, 17757, 2329, 12559, 1012, 102]


In [4]:
import evaluate
import numpy as np

metric = evaluate.load('seqeval')

def calculate_metrics(output):
    pred_batch, label_batch = output
    pred_batch = np.argmax(pred_batch, axis=2)

    pred_labels = []
    true_labels = []

    for pred, label in zip(pred_batch, label_batch):
        pred_list = []
        label_list = []
        for p, l in zip(pred, label):
            if l != -100:
                pred_list.append(label_names[p])
                label_list.append(label_names[l])
        pred_labels.append(pred_list)
        true_labels.append(label_list)
    
    result = metric.compute(predictions=pred_labels, references=true_labels)

    return {
        'recall': result['overall_recall'],
        'precision': result['overall_precision'],
        'f1': result['overall_f1'],
        'accuracy': result['overall_accuracy']
    }


In [5]:
from transformers import AutoModelForSequenceClassification



model = AutoModelForTokenClassification.from_pretrained(
    'bert-base-uncased',
    num_labels=num_labels,
    id2label=id2label,
    label2id=label2id
)

model

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12

In [6]:
from transformers import DataCollatorForTokenClassification


data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer) 

In [7]:
from transformers import Trainer, TrainingArguments, EarlyStoppingCallback


training_args = TrainingArguments(
    output_dir='./ner_output',
    logging_dir='./logs',
    save_strategy="epoch",
    eval_strategy="epoch",
    overwrite_output_dir=True,
    learning_rate=2e-5,
    num_train_epochs=20,
    load_best_model_at_end=True,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    fp16=True
)


trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['validation'],
    processing_class=tokenizer,
    compute_metrics=calculate_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

trainer.train()

Epoch,Training Loss,Validation Loss,Recall,Precision,F1,Accuracy
1,0.194900,0.053707,0.926624,0.908731,0.917590,0.985223
2,0.035500,0.051434,0.948839,0.930210,0.939432,0.987831
3,0.019100,0.048768,0.945809,0.939799,0.942795,0.988610
4,0.011900,0.055786,0.947997,0.941816,0.944896,0.988785
5,0.008000,0.056170,0.950353,0.941480,0.945896,0.988824
6,0.004400,0.061564,0.949512,0.937521,0.943478,0.988824


TrainOutput(global_step=5268, training_loss=0.03588101261025952, metrics={'train_runtime': 389.8308, 'train_samples_per_second': 720.364, 'train_steps_per_second': 45.045, 'total_flos': 2045112348578508.0, 'train_loss': 0.03588101261025952, 'epoch': 6.0})

In [13]:
test_result = trainer.evaluate(tokenized_dataset['test'])
print(test_result)

{'eval_loss': 0.12496345490217209, 'eval_recall': 0.9049220963172805, 'eval_precision': 0.8935314685314686, 'eval_f1': 0.8991907107670656, 'eval_accuracy': 0.9795412942823302, 'eval_runtime': 4.4175, 'eval_samples_per_second': 781.659, 'eval_steps_per_second': 48.896, 'epoch': 6.0}


In [ ]:
from transformers import pipeline


model = AutoModelForTokenClassification.from_pretrained('./ner_output/checkpoint-4390')

ner_pipeline = pipeline(
    task='ner',
    model=model,
    tokenizer=tokenizer,
    aggregation_strategy='first'
)

query = 'Barack Obama and Elon Musk met at Google headquarters in California yesterday.'

ner_pipeline(query)

Device set to use cuda:0


[{'entity_group': 'PER',
  'score': np.float32(0.998122),
  'word': 'barack obama',
  'start': 0,
  'end': 12},
 {'entity_group': 'PER',
  'score': np.float32(0.95636237),
  'word': 'elon musk',
  'start': 17,
  'end': 26},
 {'entity_group': 'ORG',
  'score': np.float32(0.9990522),
  'word': 'google',
  'start': 34,
  'end': 40},
 {'entity_group': 'LOC',
  'score': np.float32(0.99920386),
  'word': 'california',
  'start': 57,
  'end': 67}]